In [521]:
import pandas as pd
import re

In [522]:
hyper_method_name = "temperature1.0"

In [ ]:
gt = pd.read_csv("../output/csv/groundtruth.csv")
answer = pd.read_csv(f"../output/csv/{hyper_method_name}.csv").drop(columns="Additional treatment")

In [524]:
Morphological_Properties_col = ['Particle size', 'Particle shape', 'Particle distribution', 'Coating layer characteristics', 'Crystal structure and lattice characteristics']

In [525]:
gt = gt.drop(columns=Morphological_Properties_col)
answer = answer.drop(columns=Morphological_Properties_col)

In [526]:
def c_rate_func(x):
    if "C-rate" in x:
        return True
    else: 
        return False

In [527]:
def combine_duplicated_cols(df):
    int_crate_cols = [
        col for col in df.columns
        if re.fullmatch(r"C-rate\s+[0-9]+", col)
    ]    
    
    int2float_map = {}
    for int_col in int_crate_cols:
        int2float_map[f"{int_col}"] = int_col[:7] + f"{float(int_col[7:])}"

    for int_col, float_col in int2float_map.items():
        if float_col in df.columns:
            df[int_col] = df[int_col].combine_first(df[float_col])
            df.drop(columns=float_col, inplace=True)
        df.rename(columns={int_col: float_col}, inplace=True)
    
    return df

In [528]:
gt = combine_duplicated_cols(gt)
answer = combine_duplicated_cols(answer)

In [529]:
gt_c_rate = gt.columns[gt.columns.map(c_rate_func)].to_list()
answer_c_rate = answer.columns[answer.columns.map(c_rate_func)].to_list()

In [530]:
# gt_c_rate

In [531]:
# answer_c_rate

In [532]:
# c_rate = gt_c_rate + answer_c_rate
# set(c_rate)

In [533]:
other_c_rate_col = [
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [534]:
paper_id = gt["Paper ID"].unique()

In [535]:
gt.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,Commercial NCM used,Lithium source,Synthesis method,...,C-rate 0.05,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 0.2,W ratio,C-rate 4.0,C-rate 6.0
0,11,NR0,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,167.64,45.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,169.21,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,172.46,75.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,186.64,95.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,1,0.33,0.33,0.33,2,no,LiAc,sol-gel method,...,NaN,101.7,71.5,44.5,13.2,NaN,NaN,NaN,NaN,NaN


In [536]:
answer.head()

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 0.33,C-rate 0.3,C-rate 3.0,C-rate 4.0,C-rate 6.0
0,11,NR0,1.00,0.33,0.33,0.33,2.0,NaN,yes,LiNO3,...,45.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,1.00,0.33,0.33,0.33,2.0,NaN,yes,LiNO3,...,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,1.00,0.33,0.33,0.33,2.0,NaN,yes,LiNO3,...,75.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,1.00,0.33,0.33,0.33,2.0,NaN,yes,LiNO3,...,95.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,1.01,0.35,0.32,0.32,NaN,NaN,no,LiAc,...,101.7,71.5,44.5,13.2,NaN,NaN,NaN,NaN,NaN,NaN


In [537]:
sort_col = [
    'Paper ID',
    'Sample',
    'Li ratio',
    'Ni ratio',
    'Co ratio',
    'Mn ratio',
    'O ratio',
    'W ratio',
    'Commercial NCM used',
    'Lithium source',
    'Synthesis method',
    'Crystallization method',
    'Crystallization final temperature',
    'Crystallization final duration (hours)',
    'Doping',
    'Coating',
    'Active material to Conductive additive to Binder ratio',
    'Electrolyte salt',
    'Electrolyte concentration',
    'Electrolyte solvent',
    'Electrolyte solvent ratio',
    'Additive',
    'Loading density (mass loading of NCM)',
    'Voltage range',
    'Temperature',
    'C-rate 0.1',
    'C-rate 0.5',
    'C-rate 1.0',
    'C-rate 2.0',
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [538]:
num_col = [
    "Li ratio", 
    "Ni ratio", 
    "Co ratio", 
    "Mn ratio", 
    "O ratio", 
    "W ratio", 
    "Commercial NCM used",
    "Crystallization final temperature", 
    "Crystallization final duration (hours)", 
    "Electrolyte salt", 
    'C-rate 0.1',
    'C-rate 0.5',
    'C-rate 1.0',
    'C-rate 2.0',    
    'C-rate 0.05',
    'C-rate 5.0',
    'C-rate 10.0',
    'C-rate 20.0',
    'C-rate 40.0',
    'C-rate 0.33',
    'C-rate None',
    'C-rate 0.2',
    'C-rate 0.3',
    'C-rate 4.0',
    'C-rate 6.0',
]

In [539]:
import pandas as pd

def is_number(val):
    try:
        float(val)
        return True
    except:
        return False

# key column 정의
key_cols = ["Paper ID", "Sample"]

# 모든 열 모음 (비교 대상)
all_cols = set(gt.columns) | set(answer.columns)
compare_cols = [col for col in all_cols if col not in key_cols]

# 결과 저장
results = []

# 모든 논문 처리
all_papers = sorted(set(gt["Paper ID"]) | set(answer["Paper ID"]))

for pid in all_papers:
    gt_group = gt[gt["Paper ID"] == pid].reset_index(drop=True)
    ans_group = answer[answer["Paper ID"] == pid].reset_index(drop=True)

    max_len = max(len(gt_group), len(ans_group))

    def pad_df(df, target_len):
        pad_len = target_len - len(df)
        if pad_len <= 0:
            return df
        empty = pd.DataFrame({col: [None] * pad_len for col in df.columns})
        empty["Paper ID"] = pid
        return pd.concat([df, empty], ignore_index=True)

    gt_group = pad_df(gt_group, max_len)
    ans_group = pad_df(ans_group, max_len)

    for i in range(max_len):
        row_gt = gt_group.iloc[i]
        row_ans = ans_group.iloc[i]

        result_row = {
            "Paper ID": pid,
            "Sample": row_gt["Sample"] if pd.notna(row_gt["Sample"]) else row_ans["Sample"]
        }

        for col in compare_cols:
            val_gt = row_gt[col] if col in row_gt and col in gt.columns else None
            val_ans = row_ans[col] if col in row_ans and col in answer.columns else None

            try:
                # C-rate 열이고 둘 다 NaN이면, 결과도 NaN
                if "C-rate" in col and pd.isna(val_gt) and pd.isna(val_ans):
                    match = float('nan')
                elif pd.isna(val_gt) and pd.isna(val_ans):
                    match = True
                elif pd.notna(val_gt) and pd.notna(val_ans):
                    if is_number(val_gt) and is_number(val_ans):
                        match = abs(float(val_gt) - float(val_ans)) <= 0.001
                        
                    else:
                        match = str(val_gt).strip() == str(val_ans).strip()
                else:
                    match = False

            except:
                match = False

            # 결과 저장 방식 변경
            if col in num_col:
                result_row[col] = match
            else:
                result_row[col] = val_ans  # 숫자 컬럼이 아닌 경우는 answer 값 그대로 저장

        results.append(result_row)

# 결과 DataFrame
df_result = pd.DataFrame(results)

# 누락된 열 채우기 (혹시라도)
for col in num_col:
    if col not in df_result.columns:
        df_result[col] = False

# 컬럼 순서 정렬
df_result = df_result[sort_col]

# 저장
df_result.to_csv(f"./output/csv/acc_{hyper_method_name}.csv", index=False)

C:\Users\ghckd\AppData\Local\Temp\ipykernel_137528\2792571078.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, empty], ignore_index=True)


In [540]:
answer

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 0.33,C-rate 0.3,C-rate 3.0,C-rate 4.0,C-rate 6.0
0,11,NR0,1.00,0.330000,0.33,0.33,2.0,NaN,yes,LiNO3,...,45.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,1.00,0.330000,0.33,0.33,2.0,NaN,yes,LiNO3,...,47.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,1.00,0.330000,0.33,0.33,2.0,NaN,yes,LiNO3,...,75.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,1.00,0.330000,0.33,0.33,2.0,NaN,yes,LiNO3,...,95.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,1.01,0.350000,0.32,0.32,NaN,NaN,no,LiAc,...,101.70,71.5,44.5,13.2,NaN,NaN,NaN,NaN,NaN,NaN
5,16,NCM/C,1.03,0.320000,0.33,0.34,NaN,NaN,no,LiAc,...,135.80,109.4,81.9,54.6,NaN,NaN,NaN,NaN,NaN,NaN
6,22,pristine,0.84,0.840000,0.10,0.06,2.0,NaN,yes,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,22,V-0.005,0.84,0.839975,0.10,0.06,2.0,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22,V-0.01,0.84,0.839950,0.10,0.06,2.0,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,22,V-0.02,0.84,0.839900,0.10,0.06,2.0,NaN,no,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [541]:
gt

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,Commercial NCM used,Lithium source,Synthesis method,...,C-rate 0.05,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate None,C-rate 0.2,W ratio,C-rate 4.0,C-rate 6.0
0,11,NR0,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,167.64,45.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,169.21,47.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,172.46,75.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,1,0.33,0.33,0.33,2,no,LiNO3,solution combustion,...,186.64,95.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,1,0.33,0.33,0.33,2,no,LiAc,sol-gel method,...,NaN,101.70,71.5,44.5,13.2,NaN,NaN,NaN,NaN,NaN
5,16,NCM/C,1,0.33,0.33,0.33,2,no,LiAc,sol-gel method,...,NaN,135.80,109.4,81.9,54.6,NaN,NaN,NaN,NaN,NaN
6,22,Pristine,1,0.84,0.10,0.06,2,no,LiOH,solid state method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,22,V-0.005,1,0.84,0.10,0.06,2,no,LiOH,solid state method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22,V-0.01,1,0.84,0.10,0.06,2,no,LiOH,solid state method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,22,V-0.02,1,0.84,0.10,0.06,2,no,LiOH,solid state method,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [542]:
df_result

,Paper ID,Sample,Li ratio,Ni ratio,Co ratio,Mn ratio,O ratio,W ratio,Commercial NCM used,Lithium source,...,C-rate 5.0,C-rate 10.0,C-rate 20.0,C-rate 40.0,C-rate 0.33,C-rate None,C-rate 0.2,C-rate 0.3,C-rate 4.0,C-rate 6.0
0,11,NR0,True,True,True,True,True,True,False,LiNO3,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,NR1,True,True,True,True,True,True,False,LiNO3,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,NR3,True,True,True,True,True,True,False,LiNO3,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NR5,True,True,True,True,True,True,False,LiNO3,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,NCM,False,False,False,False,False,True,True,LiAc,...,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN
5,16,NCM/C,False,False,True,False,False,True,True,LiAc,...,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN
6,22,Pristine,False,True,True,True,True,True,False,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,22,V-0.005,False,True,True,True,True,True,True,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,22,V-0.01,False,True,True,True,True,True,True,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,22,V-0.02,False,True,True,True,True,True,True,LiOH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [543]:
# df_result = df_result.loc[df_result["Paper ID"] == 11, num_col]

In [544]:
# df_result.loc[df_result["Paper ID"] == 11, "Li ratio":].mean().mean()

In [545]:
# results = []

# # 공통 key
# key_cols = ["Paper ID", "Sample"]

# # 비교할 column들 (공통 열)
# common_cols = list(set(gt.columns) | set(answer.columns))
# common_cols = [col for col in common_cols if col not in key_cols]

# # gt와 answer 모두 정렬
# gt_sorted = gt.sort_values(by=key_cols).reset_index(drop=True)
# answer_sorted = answer.sort_values(by=key_cols).reset_index(drop=True)

# # 모든 row를 하나씩 비교
# for idx in range(len(gt_sorted)):
#     row_gt = gt_sorted.loc[idx]
#     row_pred = answer_sorted.loc[idx]
    
#     result_row = {
#         "Paper ID": row_gt["Paper ID"],
#         "Sample": row_gt["Sample"]
#     }

#     for col in common_cols:
#         if col not in gt.columns:
#             match = False
#         val_gt = row_gt[col]
#         val_pred = row_pred[col]

#         # 숫자 비교 시 오차 허용
#         try:
#             if pd.isna(val_gt) and pd.isna(val_pred):
#                 match = True
#             elif pd.notna(val_gt) and pd.notna(val_pred):
#                 if isinstance(val_gt, (int, float)) and isinstance(val_pred, (int, float)):
#                     match = abs(float(val_gt) - float(val_pred)) < 1e-3
#                 else:
#                     match = str(val_gt).strip() == str(val_pred).strip()
#             else:
#                 match = False
#         except:
#             match = False

#         result_row[col] = match

#     results.append(result_row)

# # 결과 DataFrame으로 변환
# df_result = pd.DataFrame(results)
# df_result.to_csv("comparison_result.csv", index=False)


In [546]:
# list(answer.columns)

In [547]:
# sort_col = [
#     'Paper ID',
#     'Sample',
#     'Li ratio',
#     'Ni ratio',
#     'Co ratio',
#     'Mn ratio',
#     'O ratio',
#     'W ratio',
#     'Commercial NCM used',
#     'Lithium source',
#     'Synthesis method',
#     'Crystallization method',
#     'Crystallization final temperature',
#     'Crystallization final duration (hours)',
#     'Doping',
#     'Coating',
#     'Active material to Conductive additive to Binder ratio',
#     'Electrolyte salt',
#     'Electrolyte concentration',
#     'Electrolyte solvent',
#     'Electrolyte solvent ratio',
#     'Additive',
#     'Loading density (mass loading of NCM)',
#     'Voltage range',
#     'Temperature',
#     'C-rate 0.1',
#     'C-rate 0.2',
#     'C-rate 0.5',
#     'C-rate 1.0',
#     'C-rate 2.0',
#     'C-rate 5.0',
#     'C-rate 0.05',
#     'C-rate None',
#     'C-rate 4.0',
#     'C-rate 6.0'
# ]

In [548]:
# df_result[sort_col]

In [549]:
# gt[[
#  'C-rate 10.0',
#  'C-rate 20.0',
#  'C-rate 40.0',
#  'C-rate 0.33',
# ]]

In [550]:
# answer[[
#  'C-rate 10.0',
#  'C-rate 20.0',
#  'C-rate 40.0',
#  'C-rate 0.33',
# ]]

In [551]:
# pd.DataFrame(results)

In [552]:
# gt[gt["Paper ID"] == 11]

In [553]:
# answer